In [1]:
import cv2
import numpy as np
import os
from random import shuffle
from tqdm import tqdm

In [6]:
TRAIN_DIR = '/home/tejpal/Desktop/AI_Assignment2'
TEST_DIR = '/home/tejpal/Desktop/AI_Assignment2/test'
IMG_SIZE = 50
LR = 1e-3
MODEL_NAME = 'skin_cancer-{}-{}.model'.format(LR, '2conv-basic')

In [7]:
def label_img(img):
    word_label = img.split('.')[-3]
    if word_label == 'beg': return [1,0]
    elif word_label == 'mal': return [0,1]

In [8]:
def create_train_data():
    training_data = []
    for img in tqdm(os.listdir(TRAIN_DIR)):
        label = label_img(img)
        path = os.path.join(TRAIN_DIR,img)
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        training_data.append([np.array(img),np.array(label)])
    shuffle(training_data)
    np.save('train_data.npy', training_data)
    return training_data

In [9]:
def process_test_data():
    testing_data = []
    for img in tqdm(os.listdir(TEST_DIR)):
        path = os.path.join(TEST_DIR,img)
        img_num = img.split('.')[0]
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        testing_data.append([np.array(img), img_num])
        
    shuffle(testing_data)
    np.save('test_data.npy', testing_data)
    return testing_data

In [10]:
train_data = create_train_data()

100%|██████████████████████████████████████| 2000/2000 [06:22<00:00,  5.23it/s]


In [12]:
import tensorflow as tf
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
tf.reset_default_graph()
convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input')

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 128, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 2, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')


hdf5 is not supported on this machine (please install/reinstall h5py for optimal experience)
curses is not supported on this machine (please install/reinstall curses for an optimal experience)
Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [13]:
if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('model loaded!')

INFO:tensorflow:Restoring parameters from C:\Python35\Scripts\skin_cancer-0.001-2conv-basic.model
model loaded!


In [14]:
train = train_data[:-500]
test = train_data[-500:]


In [15]:
X = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
Y = [i[1] for i in train]

test_x = np.array([i[0] for i in test]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
test_y = [i[1] for i in test]




In [16]:
model.fit({'input': X}, {'targets': Y}, n_epoch=20, validation_set=({'input': test_x}, {'targets': test_y}), 
    snapshot_step=500, show_metric=True, run_id=MODEL_NAME)

model.save(MODEL_NAME)

Training Step: 2399  | total loss: 0.14737 | time: 7.416s
| Adam | epoch: 020 | loss: 0.14737 - acc: 0.9577 -- iter: 1472/1500
Training Step: 2400  | total loss: 0.14132 | time: 8.748s
| Adam | epoch: 020 | loss: 0.14132 - acc: 0.9588 | val_loss: 0.35682 - val_acc: 0.8820 -- iter: 1500/1500
--
INFO:tensorflow:C:\Python35\Scripts\skin_cancer-0.001-2conv-basic.model is not in all_model_checkpoint_paths. Manually adding it.


In [18]:
import matplotlib.pyplot as plt
test_data = process_test_data()
print("Prediction Result:")
for num,data in enumerate(test_data[:20]):
    # benign: [1,0]
    # malignant: [0,1]
    
    img_num = data[1]
    img_data = data[0]
    orig = img_data
    data = img_data.reshape(IMG_SIZE,IMG_SIZE,1)
    #model_out = model.predict([data])[0]
    model_out = model.predict([data])[0]
    
    if np.argmax(model_out) == 1: str_label='malignant'
    else: str_label='benign'
        
    


100%|██████████████████████████████████████████| 20/20 [00:00<00:00, 52.73it/s]


Prediction Result:


In [28]:

Y_true=[i[1] for i in test]



In [29]:
Y_pred=[]
for num,data in enumerate(test):
    img_data = data[0]
    orig = img_data
    data = img_data.reshape(IMG_SIZE,IMG_SIZE,1)
    model_out = model.predict([data])[0]
    Y_pred.append(model_out)
for data in (Y_pred):
    if data[0]>0.5: data[0]=int(1)
    else: data[0]=int(0)
    if data[1]>0.5: data[1]=int(1)
    else: data[1]=int(0)
   

In [32]:
from sklearn.metrics import confusion_matrix
y_true=[]
y_pred=[]
for data in (Y_true):
    if(data[0]==1 and data[1]==0):
        y_true.append(1)
    else: y_true.append(0)

for data in (Y_pred):
    if(data[0]==1 and data[1]==0):
        y_pred.append(1)
    else: y_pred.append(0)
        

cm=confusion_matrix(y_true, y_pred)


In [33]:
print("Confusion Matrix of Melonoma")
print("          Benign | Malignant")
for i in (1,0):
    if i==1: print("Benign   |",end=' ')
    else: print("Malignant|",end=' ') 
    for j in (1,0):
        print(cm[i][j],end='      ') 
    print()

Confusion Matrix of Melonoma
          Benign | Malignant
Benign   | 350      63      
Malignant| 41      46      


In [34]:
a=cm[1][1]
b=cm[1][0]
c=cm[0][1]
d=cm[0][0]


In [35]:
print("Precision and Recall for Benign Class")
print("Precision:",end=' ')
print(a/(a+c))
print("Recall:",end=' ')
print(a/(a+b))


Precision and Recall for Benign Class
Precision: 0.714929577464789
Recall: 0.7043099273607748


In [36]:
print("Precision and Recall for Malignant Class")
print("Precision:",end=' ')
print(b/(b+d))
print("Recall:",end=' ')
print(b/(a+b))


Precision and Recall for Malignant Class
Precision: 0.3108108108108108
Recall: 0.05569007263922518
